# Cantilever beam

This is a simple example for simulating a cantilever beam using Python and scikit-fem.

# Mesh creation

First we create a mesh for the domain $(0, 5) \times (0, 1)^2$ and use hexahedral elements.

In [17]:
import numpy as np

from skfem import *

m = MeshHex.init_tensor(
    np.linspace(0, 5, 15),
    np.linspace(0, 1, 5),
    np.linspace(0, 1, 5),
)

# use trilinear hexahedra
e = ElementVectorH1(ElementHex1())

# Build stiffness matrix

Next we build the system matrix (aka. stiffness matrix in the context of elasticity).

In [18]:
from skfem.models.elasticity import linear_elasticity, lame_parameters

basis = InteriorBasis(m, e)

# set material parameters, Young's modulus and Poisson ratio
K = linear_elasticity(*lame_parameters(1e9, 0.3)).assemble(basis)

# Build load vector

Set downward loading at $x=5$.

In [19]:
facet_basis = FacetBasis(m, e, facets=m.facets_satisfying(lambda x: x[0] == 5.0))

@LinearForm
def linf(v, w):
    # 10 N / m^2
    return 10 * v.value[1]

f = linf.assemble(facet_basis)

# Solve displacements

Also set displacement to zero at $x=0$.

In [20]:
x = solve(*condense(K, f, D=basis.get_dofs(lambda x: x[0] == 0.0)))

# Calculate $xx$-component of strain tensor

Calculates its mean value in each element.

In [21]:
@Functional
def strain(w):
    from skfem.helpers import sym_grad
    return sym_grad(w['disp'])[0, 0]

exx = strain.elemental(basis, disp=basis.interpolate(x))

# Displace mesh and save to vtk

In [22]:
for i in [0, 1, 2]:
    m.p[i] += 2e5 * x[basis.nodal_dofs[i]]  # scaling factor 1e5
m.save('beam.vtk', point_data={}, cell_data={'exx': exx})

# Visualize in Paraview

![Visualization in Paraview](beam.png)